<b>Вариант 15</b>

<b>1. Для заданного пользователя рассчитать оценки для всех
фильмов, которые он не оценил.</b>

In [1]:
import pandas as pd
import numpy as np
import math
import json
from collections import defaultdict

In [2]:
#User 15 будет находиться на строке 14
user15 = 15
v = user15 - 1
k = 4
no_rating = -1

In [3]:
#Считываем файлы
data = pd.read_csv('data.csv', delimiter = ', ', engine = 'python')
context_day = pd.read_csv('context_day.csv', delimiter = ', ', engine = 'python')
context_place = pd.read_csv('context_place.csv', delimiter = ', ', engine = 'python')
movie_names = pd.read_csv('movie_names.csv', delimiter = ', ', engine = 'python')

In [4]:
#Удаляем столбец с юзерами
data = data.drop(data.columns[0], axis = 1)

In [5]:
data

,Movie 1,Movie 2,Movie 3,Movie 4,Movie 5,Movie 6,Movie 7,Movie 8,Movie 9,Movie 10,...,Movie 21,Movie 22,Movie 23,Movie 24,Movie 25,Movie 26,Movie 27,Movie 28,Movie 29,Movie 30
0,-1,4,1,5,-1,-1,-1,1,3,-1,...,2,4,-1,-1,3,4,-1,2,4,1
1,4,5,2,3,3,4,-1,5,3,2,...,-1,2,-1,3,1,5,-1,5,2,2
2,4,2,5,4,1,5,-1,3,5,2,...,4,-1,5,4,5,1,2,1,-1,3
3,5,-1,2,-1,-1,2,4,3,2,3,...,-1,2,2,2,2,5,3,2,5,5
4,4,3,3,3,5,3,3,4,5,2,...,4,5,2,-1,3,5,1,-1,5,-1
5,2,-1,3,3,3,3,4,2,-1,1,...,5,-1,1,3,-1,4,4,5,1,2
6,4,1,4,4,1,4,5,2,2,-1,...,2,5,1,3,4,-1,4,3,1,4
7,-1,5,5,3,3,3,3,3,1,2,...,-1,3,1,3,2,1,4,-1,1,3
8,1,2,3,1,5,3,3,1,2,5,...,3,2,3,-1,1,4,3,1,5,3
9,5,-1,1,-1,3,3,4,1,1,2,...,4,1,4,1,4,5,2,3,-1,1


In [6]:
sim = np.zeros(data.shape[0])

In [7]:
#В качестве метрики сходства используем метрику косинуса
for u in range(data.shape[0]):
    if (u != v):
        sumuv = 0
        sumu2 = 0
        sumv2 = 0
        uv = data.iloc[[u, v]]
        uv = uv.transpose()
        for i,movie in uv.iterrows():
            if ((movie[u] != no_rating) & (movie[v] != no_rating )):
                sumuv += movie[u] * movie[v]
                sumu2 += pow(movie[u], 2)
                sumv2 += pow(movie[v], 2)
        sim[u] = round(sumuv / ((math.sqrt(sumu2) * math.sqrt(sumv2))), 3)

In [8]:
sim

array([0.892, 0.827, 0.803, 0.932, 0.845, 0.816, 0.769, 0.857, 0.845,
       0.848, 0.838, 0.852, 0.834, 0.907, 0.   , 0.865, 0.809, 0.851,
       0.803, 0.827, 0.789, 0.829, 0.854, 0.887, 0.763, 0.814, 0.884,
       0.848, 0.857, 0.871, 0.824, 0.838, 0.846, 0.86 , 0.813, 0.861,
       0.892, 0.864, 0.793, 0.795])

In [9]:
#Выбираем 4 наиболее похожих пользователя
sim4 = np.argpartition(sim, -k)[-k:]

In [10]:
sim4

array([36, 13,  3,  0], dtype=int32)

In [11]:
#Средняя оценка у пользователя user
def means(user):
    mylist = []
    for i in range(data.shape[1]):
        if data.iloc[user][i] != no_rating:
            mylist.append(data.iloc[user][i])
    arr = np.array(mylist)
    return np.mean(arr)

In [12]:
#Массив неоцененных фильмов user 15
mylist = []
for i in range(data.shape[1]):
    if (data.iloc[v][i] == no_rating):
        mylist.append(i)
unrated_films = np.array(mylist)

unrated_films

array([ 5, 10, 11, 19, 26, 27])

In [13]:
#Расчет оценки rui пользователя
dict_rates = {}
for film in unrated_films:
    sumsimr = 0
    sumsim = 0
    for u in sim4:
        if (data.iloc[u, film] != no_rating):
            sumsimr += sim[u] * (data.iloc[u, film] - means(u))
            sumsim += sim[u]
    dict_rates[film] = round(means(v) + sumsimr / sumsim, 3)
    
dict_rates

{5: 2.898, 10: 3.308, 11: 3.308, 19: 2.299, 26: 2.918, 27: 2.284}

In [14]:
#Ответ в более понятном виде
rates = {}
for i in dict_rates.keys():
    rates['Movie '+str(i+1)] = dict_rates[i]

In [15]:
rates

{'Movie 6': 2.898,
 'Movie 11': 3.308,
 'Movie 12': 3.308,
 'Movie 20': 2.299,
 'Movie 27': 2.918,
 'Movie 28': 2.284}

<b>2. Порекомендовать заданному пользователю 1
фильм, который он посмотрел бы в выходной дома,
если такой фильм есть в данных.</b>

In [16]:
#Удаляем столбец с юзерами
context_day = context_day.drop(context_day.columns[0], axis = 1)

In [17]:
context_day

,Movie 1,Movie 2,Movie 3,Movie 4,Movie 5,Movie 6,Movie 7,Movie 8,Movie 9,Movie 10,...,Movie 21,Movie 22,Movie 23,Movie 24,Movie 25,Movie 26,Movie 27,Movie 28,Movie 29,Movie 30
0,-1,Thu,Tue,Mon,-1,-1,-1,Tue,Sun,-1,...,Thu,Thu,-1,-1,Sun,Sat,-1,Wed,Wed,Thu
1,Mon,Wed,Mon,Sat,Sat,Thu,-1,Mon,Fri,Thu,...,-1,Sun,-1,Mon,Mon,Fri,-1,Sun,Tue,Wed
2,Tue,Wed,Sat,Tue,Sat,Mon,-1,Thu,Wed,Thu,...,Tue,-1,Wed,Thu,Tue,Thu,Wed,Wed,-1,Sun
3,Mon,-1,Wed,-1,-1,Sun,Sat,Mon,Mon,Wed,...,-1,Sun,Fri,Sat,Thu,Sat,Fri,Fri,Sat,Sun
4,Thu,Fri,Wed,Fri,Mon,Thu,Sat,Tue,Thu,Fri,...,Fri,Fri,Mon,-1,Wed,Mon,Sat,-1,Wed,-1
5,Mon,-1,Tue,Sat,Fri,Fri,Mon,Sun,-1,Thu,...,Thu,-1,Wed,Wed,-1,Wed,Sun,Thu,Sat,Mon
6,Thu,Tue,Sat,Fri,Sun,Sun,Fri,Wed,Mon,-1,...,Tue,Sun,Sun,Tue,Wed,-1,Fri,Tue,Wed,Wed
7,-1,Sun,Fri,Sat,Fri,Thu,Thu,Tue,Sat,Wed,...,-1,Sat,Fri,Tue,Fri,Sun,Thu,-1,Sun,Wed
8,Sat,Sun,Thu,Thu,Tue,Wed,Mon,Tue,Sat,Mon,...,Wed,Wed,Tue,-1,Fri,Thu,Fri,Wed,Sun,Mon
9,Sun,-1,Tue,-1,Mon,Fri,Thu,Tue,Thu,Fri,...,Thu,Tue,Sun,Thu,Thu,Sun,Sat,Tue,-1,Mon


In [18]:
#Удаляем столбец с юзерами
context_place = context_place.drop(context_place.columns[0], axis = 1)

In [19]:
context_place

,Movie 1,Movie 2,Movie 3,Movie 4,Movie 5,Movie 6,Movie 7,Movie 8,Movie 9,Movie 10,...,Movie 21,Movie 22,Movie 23,Movie 24,Movie 25,Movie 26,Movie 27,Movie 28,Movie 29,Movie 30
0,-1,h,h,c,-1,-1,-1,v,h,-1,...,h,c,-1,-1,c,c,-1,v,h,c
1,v,v,v,v,h,c,-1,v,h,c,...,-1,c,-1,c,h,v,-1,v,v,c
2,v,h,c,c,c,c,-1,h,c,v,...,v,-1,v,v,h,h,h,v,-1,v
3,v,-1,h,-1,-1,h,v,v,c,h,...,-1,v,h,h,c,v,c,v,v,c
4,c,c,c,c,v,h,v,v,c,h,...,v,c,v,-1,h,c,v,-1,h,-1
5,c,-1,c,h,v,c,v,v,-1,c,...,h,-1,c,h,-1,h,h,h,c,c
6,v,h,v,c,c,v,c,c,v,-1,...,h,c,c,c,v,-1,v,h,h,c
7,-1,h,c,h,h,v,h,h,v,c,...,-1,v,c,v,h,h,c,-1,h,c
8,h,c,h,v,h,h,v,c,c,h,...,c,v,h,-1,v,h,v,c,h,h
9,v,-1,v,-1,v,c,h,c,h,v,...,c,v,h,v,c,v,v,h,-1,c


In [20]:
#фильмы, просмотренные дома
home = ['h']
#фильмы, просмотренные на выходных
weekends = ['Sun','Sat']

In [21]:
#Для того, чтобы порекомендовать фильм, смотрим на то, как часто каждый фильм смотрелся дома и в выходные
#home_ind - как часто фильм смотрелся дома
#weekends_ind - как часто фильм смотрелся в выходные
_default_data = lambda: defaultdict(_default_data)
data_films = _default_data()
for film in unrated_films:
    home_ind = 0 #просмотры фильма дома
    weekends_ind = 0 #просмотры фильма на выходных
    total = 0 #просмотры фильма
    for user in range(data.shape[0]):
        if (context_place.iloc[user][film] in home):
            home_ind += 1
        if (context_day.iloc[user][film] in weekends):
            weekends_ind += 1
        if (context_place.iloc[user][film] != no_rating):
            total += 1
    data_films[int(film)]['home_ind'] = round(home_ind / total, 3)
    data_films[int(film)]['weekends_ind'] = round(weekends_ind / total, 3)  
    data_films[int(film)]['rate'] = dict_rates[film]

In [22]:
films = {}
for i in dict_rates.keys():
    films['Movie '+str(i+1)] = data_films[i]

In [23]:
#получаем словарь с фильмом и его индексом просмотра дома, индексом просмотра на выходных, рейтингом
print(json.dumps(films, indent = 2)) 

{
  "Movie 6": {
    "home_ind": 0.2,
    "weekends_ind": 0.125,
    "rate": 2.898
  },
  "Movie 11": {
    "home_ind": 0.2,
    "weekends_ind": 0.225,
    "rate": 3.308
  },
  "Movie 12": {
    "home_ind": 0.3,
    "weekends_ind": 0.2,
    "rate": 3.308
  },
  "Movie 20": {
    "home_ind": 0.325,
    "weekends_ind": 0.15,
    "rate": 2.299
  },
  "Movie 27": {
    "home_ind": 0.225,
    "weekends_ind": 0.3,
    "rate": 2.918
  },
  "Movie 28": {
    "home_ind": 0.25,
    "weekends_ind": 0.25,
    "rate": 2.284
  }
}


In [24]:
#Считаем total_rate, который получается из суммы rate, home_ind, weekends_ind
total_rate = 0
for film in films.keys():
    total_rate = films[film]['home_ind'] + films[film]['weekends_ind'] + films[film]['rate']
    films[film]['total_rate'] = round(total_rate, 3)

In [25]:
print(json.dumps(films, indent = 2)) 

{
  "Movie 6": {
    "home_ind": 0.2,
    "weekends_ind": 0.125,
    "rate": 2.898,
    "total_rate": 3.223
  },
  "Movie 11": {
    "home_ind": 0.2,
    "weekends_ind": 0.225,
    "rate": 3.308,
    "total_rate": 3.733
  },
  "Movie 12": {
    "home_ind": 0.3,
    "weekends_ind": 0.2,
    "rate": 3.308,
    "total_rate": 3.808
  },
  "Movie 20": {
    "home_ind": 0.325,
    "weekends_ind": 0.15,
    "rate": 2.299,
    "total_rate": 2.774
  },
  "Movie 27": {
    "home_ind": 0.225,
    "weekends_ind": 0.3,
    "rate": 2.918,
    "total_rate": 3.443
  },
  "Movie 28": {
    "home_ind": 0.25,
    "weekends_ind": 0.25,
    "rate": 2.284,
    "total_rate": 2.784
  }
}


In [26]:
#Находим фильм с максимальным total_rate
maximum = -1
for film in films.keys():
    if (maximum < films[film]['total_rate']):
        maximum = films[film]['total_rate']
        rec_film = film
print(rec_film)

Movie 12


In [27]:
recommended_film = {}
recommended_film[rec_film] = maximum

In [28]:
#Фильм для рекомендации с новым рейтингом
recommended_film

{'Movie 12': 3.808}

In [29]:
#Записываем результат в json
result = {
    'user': user15,
    '1': rates,
    '2': recommended_film
}

with open('result.json', 'w') as file:
    json.dump(result, file, indent = 2)
    
print(json.dumps(result, indent = 2)) 

{
  "user": 15,
  "1": {
    "Movie 6": 2.898,
    "Movie 11": 3.308,
    "Movie 12": 3.308,
    "Movie 20": 2.299,
    "Movie 27": 2.918,
    "Movie 28": 2.284
  },
  "2": {
    "Movie 12": 3.808
  }
}


### Задание 3.2

In [30]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import requests

In [31]:
movieNames = pd.read_csv("movie_names.csv", header=None, delimiter=', ', index_col=0, engine = 'python')
movieNames

,1
0,
Movie 1,The Shawshank Redemption
Movie 2,The Godfather
Movie 3,The Godfather: Part II
Movie 4,The Dark Knight
Movie 5,12 Angry Men
Movie 6,Schindler's List
Movie 7,The Lord of the Rings: The Return of the King
Movie 8,Pulp Fiction
Movie 9,The Good the Bad and the Ugly


In [32]:
movieName = movieNames[movieNames.index == rec_film][1][0]
movieName

'Forrest Gump'

In [33]:
#Recommended movie: Forrest Gump (Q134773)
#Cписок фильмов, которые стоили более чем в 2 раза дороже, чем рекомендованный фильм (myFilmCost: 55000000)

API_ENDPOINT = "https://www.wikidata.org/w/api.php"
params = {
    'action' : 'wbsearchentities',
    'format' : 'json',
    'language' : 'en',
    'search': movieName
}
res = requests.get(API_ENDPOINT, params = params)
filmQ = res.json()['search'][0]['id']
filmQ

'Q134773'

In [34]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql_query = """

SELECT ?item ?itemLabel ?cost
WHERE {
  ?item wdt:P31 wd:Q11424;
        wdt:P2130 ?cost.
  ?myFilm wdt:P31 wd:Q11424 filter(?myFilm = wd:""" + filmQ + """).
  ?myFilm wdt:P1476 ?myFilmName;
          wdt:P2130 ?myFilmCost.
  FILTER(?cost > 2 * ?myFilmCost)
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
ORDER BY (?cost)

"""

sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [35]:
results_df = pd.json_normalize(results['results']['bindings'])
results_df[['item.value', 'itemLabel.value', 'cost.value']].head(results_df.shape[0])

,item.value,itemLabel.value,cost.value
0,http://www.wikidata.org/entity/Q80959,Elysium,115000000
1,http://www.wikidata.org/entity/Q110397,True Lies,115000000
2,http://www.wikidata.org/entity/Q181069,Star Wars: Episode II – Attack of the Clones,115000000
3,http://www.wikidata.org/entity/Q1145275,Windtalkers,115000000
4,http://www.wikidata.org/entity/Q914247,Dante's Peak,116000000
...,...,...,...
221,http://www.wikidata.org/entity/Q7117022,PK,1220000000
222,http://www.wikidata.org/entity/Q20968495,Viking,1250000000
223,http://www.wikidata.org/entity/Q20965600,Necrofobia,1275898738
224,http://www.wikidata.org/entity/Q37140610,Pappa pia,1380000000
